In [1]:
#find the amount of cpg pairs there are within a string of 150 bases in the
#sequence after methylation and return true if the sequence has at least 6 cpg
#pairs
#The input will be a sequence and the start of the section you are looking at
#for potential primers
def dense(newseq,start,first):
  #print(start)
  cgdense = 0
  p = -1
  if (start + 150 >= len(newseq)):
    return -1
  for i in range (start,start + 150):
    if ((newseq[i] == 'C' or newseq[i] == 'c') and i != 149):
      cgdense += 2;
  if (first and cgdense >= 30):
    return 0
  else:
    for i in range (start,len(newseq) - 150):
      #print(i,cgdense)
      if ((newseq[i] == 'C' or newseq[i] == 'c')):
        cgdense -= 2
      if ((newseq[i + 150] == 'C' or newseq[i + 150] == 'c')):
        cgdense += 2
      if (cgdense >= 12):
        #print(newseq[i:i + 150])
        #for a in range (150):
        #  fout.write(newseq[i + a + 1])
        p = i
        break
  if (p + 150 >= len(newseq) ):
    return 0-1
  return p
#the function will return the first place in the sequence after the start
#it finds has enough cpg pairs to serve as an amplicon

In [2]:
#counts and returns the number of c and g in the given sequence
#ACAGTCAGCTACGATCAGCATCAGCT will return 13
def gcdensity(seq):
  gc = 0
  #for i in range (len(seq) - 1):
  #  if (seq[i:i + 2] == "GC" or seq[i:i + 2] == "gc"):
  #    gc += 1;
  for i in range (len(seq)):
    if (seq[i] == 'G' or seq[i] == 'g' or seq[i] == 'C' or seq[i] == 'c'):
      gc += 1
  return gc

In [3]:
#Finds all pairs of two cgs in a sequence and considers
#every combination of forward and reverse primer
#that could be formed in the sequence and scores them by
#how likely they would work in an assay before returning

#The input is a sequence of 150 bases that could be a possible amplicon
#The output is a list of possible amplicons
#in the given sequence with scores to indicate
#how likely they will contain a forward and reverse primer
def cgcgpairs(newseq):
  points = [];
  #record the location of every cpg
  for i in range (len(newseq)):
    if (newseq[i] == 'C' or newseq[i] == 'c'):
      points.append(i)
      #print(i)
  start = -1
  at = 0
  pairs = []
  #records the location of pairs of cpgs
  for i in range(len(points) - 1):
    #print(points[i + 1] - points[i])
    if (points[i + 1] - points[i] <= 3) :
      pairs.append([points[i],points[i + 1]])
      #start = points[i]
      #at = i
      #break
  #end = -1
  #for i in range(len(points) - 1,at,-1):
    #print(points[i] - points[i - 1])
  #  if (points[i] - points[i - 1] <= 3) :
  #    end = points[i]
  #    break
  #if(end - start >= 100):
  #  return "-1"

  cgcgstr = ""
  #cgcgstr += str(start) + " " + str(end)
  if (len(pairs) == 1):
    return "-1"
  min = 100
  global score
  score = 0
  f = -1
  s = -1
  #for i in range(len(pairs)):
    #print(pairs[i][0])
  for i in range(len(pairs) - 1):
    #filters out primers with a cg pair too close to the start or end
    if (pairs[i][0] < 21):
      continue
    #print(pairs[i][0])
    for a in range(i + 1,len(pairs) - 1):
      if (pairs[a][1] + 1 >= len(newseq) - 20):
        continue
      #give scores based on how many bases are between the first and last pair of cpgs
      #used to punish cases of consecutive cpgs
      curscore = 0
      curscore += 5 + pairs[i][1] - pairs[i][0]
      curscore += 5 + pairs[a][1] - pairs[a][0]
      #make sure the distance between the first and last pair of cpgs is in the range of 40-100 bases
      #used to make sure forward and reverse primer are placed at an appropriate range
      if (pairs[a][1] - pairs[i][0] < 100 and pairs[a][0] - (pairs[i][1] + 1) > 40):
        sum = pairs[a][1] - pairs[i][0]
        #give scores to the primers based on how many bases they are seperated by score decreases by 1 every 20
        curscore += 6 - (sum -(sum % 20)) / 20
        if (curscore > score):
          #the newseq cannot be composed of more than 75% c or g
          if (gcdensity(newseq) > 3*len(newseq) / 4):
            continue
          #if (gcdensity(newseq[0:pairs[i][0] + 1]) > 3 * (pairs[i][0] + 1) / 4):
          #  continue
          #if (gcdensity(newseq[pairs[i][0]:pairs[a][1] + 2]) > 3 * len(newseq) / 4):
          #  continue
          #if (gcdensity(newseq[pairs[a][1] + 2:150]) > 3 * (150 - (pairs[a][1] + 2)) / 4):
          #  continue
          score = curscore
          min = pairs[a][1] - pairs[i][0]
          f = i
          s = a
          continue
        #only takes the highest scoring pair for an amplicon
        if (curscore < score):
          continue
        #every test the distance between the first and last pair is less
        if (pairs[a][1] - pairs[i][0] > min):
          break
        #print(pairs[a][1] - pairs[i][0])
        min = pairs[a][1] - pairs[i][0]
        f = i
        s = a
  #print(pairs[f][0] + 1,pairs[s][0] + 1)
  #print(f,s)
  #print(f,s)
  if (f == -1 or s == -1):
    return "-1"
  #print(newseq[pairs[f][0]:pairs[s][1] + 2])
  cgcgstr = str(pairs[f][1] + 1) + " " + str(pairs[s][0] + 2)
  #print(cgcgstr)
  #if (gcdensity(newseq[pairs[f][0]:pairs[s][1] + 2]) > 7 * len(newseq) / 10):
    #return "-1"
  #record information about the location of the section in the whole strand
  global splace
  splace = pairs[f][0]
  global eplace
  eplace = pairs[s][1]
  global splace2
  splace2 = pairs[f][1]
  global eplace2
  eplace2 = pairs[s][0]
  #print(newseq[start:end + 2])
  return cgcgstr


In [4]:
#when given a strand returns the sequence of the complementary strand
def complementary(seq):
  newseq = ""
  for i in range (len(seq)-1,-1,-1):
    if (seq[i] == 'G' or seq[i] == 'g'):
      newseq += 'C'
    if (seq[i] == 'C' or seq[i] == 'c'):
      newseq += 'G'
    if (seq[i] == 'T' or seq[i] == 't'):
      newseq += 'A'
    if (seq[i] == 'A' or seq[i] == 'a'):
      newseq += 'T'
  return newseq

In [5]:
#when given a strand returns the sequence of the complementary strand in reverse
def reverse(seq):
  newseq = ""
  for i in range (len(seq)):
    if (seq[i] == 'G' or seq[i] == 'g'):
      newseq += 'C'
    if (seq[i] == 'C' or seq[i] == 'c'):
      newseq += 'G'
    if (seq[i] == 'T' or seq[i] == 't'):
      newseq += 'A'
    if (seq[i] == 'A' or seq[i] == 'a'):
      newseq += 'T'
  return newseq

In [18]:
#flip a strand for reverse primers
def flip(seq):
  newseq = ""
  for i in range (len(seq)-1,-1,-1):
    if (seq[i] == 'G' or seq[i] == 'g'):
      newseq += 'G'
    if (seq[i] == 'C' or seq[i] == 'c'):
      newseq += 'C'
    if (seq[i] == 'T' or seq[i] == 't'):
      newseq += 'T'
    if (seq[i] == 'A' or seq[i] == 'a'):
      newseq += 'A'
  return newseq

In [25]:
#finds if two strands have primer dimers between each other

#input is two strands and an integer to indicate whether each strand is forward or reverse
def diff(forw,rev,fr):
  #print(forw,rev)
  #rep = complementary(seq)
  #print(forw)
  #print(rev)
  dimers = []
  for i in range (2,min(len(forw),len(rev))):
    for a in range (len(forw) - i + 1):
      for e in range (len(rev) - i + 1):
        #print (seq[a:i + a])
        if (forw[a:i + a] == rev[e:i + e]):
          if (fr != 3):
            dimers.append([forw[a:i + a],len(forw) - a - i,e])
          else:
            dimers.append([forw[a:i + a],a,len(rev) - e - i])
          #print(forw[a:i + a],len(forw) - a - i,e)
          #print(len(forw) - a - i,e)
  bad = False
  for i in range(len(dimers) - 1,-1,-1):
    if (dimers[i][1] != 0 and dimers[i][2] != 0):
      continue
    if (fr == 1 and (dimers[i][1] == 0 or dimers[i][2] == 0)):
      if (len(dimers[i][0]) <= 2 or dimers[i][1] > 2 or dimers[i][2] > 2):
        continue
      print(forw)
      spaces = ""
      for a in range (len(forw) - len(dimers[i][0])):
        spaces += " "
      if (dimers[i][1] > 0):
        spaces = spaces[0:len(spaces) - dimers[i][1]]
      connect = spaces
      for a in range (len(dimers[i][0])):
        connect += "|"
      if (dimers[i][2] > 0):
        spaces = spaces[0:len(spaces) - dimers[i][2]]
      print(connect)
      print(spaces + reverse(rev))
      bad = True
    elif (fr == 2 and (dimers[i][1] == 0 or dimers[i][2] == 0)):
      #print(dimers[i][0],dimers[i][1],dimers[i][2])
      if (len(dimers[i][0]) <= 2 or dimers[i][1] > 2 or dimers[i][2] > 2):
        continue
      print(forw)
      spaces = ""
      for a in range (len(forw) - len(dimers[i][0])):
        spaces += " "
      if (dimers[i][1] > 0):
        spaces = spaces[0:len(spaces) - dimers[i][1]]
      connect = spaces
      for a in range (len(dimers[i][0])):
        connect += "|"
      if (dimers[i][2] > 0):
        spaces = spaces[0:len(spaces) - dimers[i][2]]
      print(connect)
      print(spaces + reverse(rev))
      bad = True
    elif (fr == 3 and (dimers[i][1] == 0 or dimers[i][2] == 0)):
      #print(dimers[i][0],dimers[i][1],dimers[i][2])
      if (len(dimers[i][0]) <= 2 or dimers[i][1] > 2 or dimers[i][2] > 2):
        continue
      #print(dimers[i][0],dimers[i][1],dimers[i][2])
      spaces = ""
      for a in range (len(rev) - len(dimers[i][0])):
        spaces += " "
      if (dimers[i][1] > 0):
        spaces = spaces[0:len(spaces) - dimers[i][1]]
      if (dimers[i][2] > 0):
        spaces = spaces[0:len(spaces) - dimers[i][2]]
      connect = spaces
      for a in range (len(dimers[i][0])):
        connect += "|"
      print(spaces + forw)
      print(connect)
      print(reverse(rev))
      bad = True
    return bad

In [24]:
#primer dimer
#input all the forward primers and then all the reverse primers
#primers start with 5' and end with 3'
#takes in a list of primers and returns the ones that may have primer dimers
#with each other or by themselves
fin = open ('sequence.txt', 'r')
x = fin.readlines()
for i in range (len(x)):
  if (i != len(x) - 1):
    x[i] = (x[i])[0:len(x[i]) - 1]
for i in range (len(x)):
  for a in range (i,len(x)):
    if (i < len(x) / 2 and a < len(x) / 2):
      print(str(i + 1) + " forward and " + str(a + 1) + " forward:")
    elif (i < len(x) / 2 and a >= len(x) / 2):
      print(str(i + 1) + " forward and " + str(int(a - len(x) / 2) + 1) + " reverse:")
    else:
      print(str(int(i - len(x) / 2) + 1) + " reverse and " + str(int(a - len(x) / 2) + 1) + " reverse:")
    if (i == a):
      continue
    #  if (i < len(x) / 2):
    #    same(x[i],True)
    #  else:
    #    same(complementary(x[i]),False)
    else:
      if (i < len(x) / 2 and a < len(x) / 2):
        diff(x[i],flip(reverse(x[a])),1)
      elif (i < len(x) / 2 or a < len(x) / 2):
        if (diff(x[i],flip(reverse(x[a])),2)):
          bad[a - int(len(x) / 2)].append(1)
        else:
          bad[a - int(len(x) / 2)].append(0)
      else:
        diff(x[i],flip(reverse(x[a])),1)
#for i in range (8):
  #print(bad[i])


1 forward and 1 forward:
1 forward and 2 forward:
1 forward and 3 forward:
1 forward and 4 forward:
1 forward and 5 forward:
1 forward and 6 forward:
1 forward and 1 reverse:
1 forward and 2 reverse:
1 forward and 3 reverse:
1 forward and 4 reverse:
1 forward and 5 reverse:
1 forward and 6 reverse:
2 forward and 2 forward:
2 forward and 3 forward:
2 forward and 4 forward:
GGGAATATGCGGCGCGC
             ||||
             CGCGGGATGAGCGAGTGATAT
2 forward and 5 forward:
GGGAATATGCGGCGCGC
             ||||
             CGCGTTGGCGGTTTTATTTT
2 forward and 6 forward:
GGGAATATGCGGCGCGC
             ||||
             CGCGAGATGTTGGCGTTTG
2 forward and 1 reverse:
2 forward and 2 reverse:
GGGAATATGCGGCGCGC
            |||||
            GCGCGCAGCAAATCCCAC
2 forward and 3 reverse:
2 forward and 4 reverse:
GGGAATATGCGGCGCGC
            ||||
            GCGCAAATCAAAGCGCACAG
2 forward and 5 reverse:
GGGAATATGCGGCGCGC
            ||||
            GCGCAGCCAACCCAAAA
2 forward and 6 reverse:
3 forward and 3

In [ ]:
#filter out pairs with bad orderings of g
def gcheck(sseq):
  g3 = []
  g4 = False
  for i in range (20,130):
    if (sseq[i:i + 4] == 'gggg' or sseq[i:i + 4] == "GGGG"):
      g4 = True
      break
    if (sseq[i:i + 3] == "ggg" or sseq[i:i + 3] == "GGG"):
      g3.append(i)
  for i in range (len(g3) - 1):
    if (g3[i + 1] - g3[i] <= 10):
      g4 = True
      break;
  if(g4):
    return True
  return False

#input format for every set of primers and probe:
#forward primer
#probe
#reverse primer
#forward primer temperature
#probe temperature
#reverse primer temperature

fin = open ('sequence.txt', 'r')
fout = open ('newsequence.txt', 'w')
x = fin.readlines()
#post primer3 checking
for a in range (len(x) / 6):
  primerf = x[a * 6]
  probe = x[a * 6 + 1]
  primerr = x[a * 6 + 2]
  if (gcheck(primerf)) :
    continue
  #Probe 5' end cannot be a G
  if (probe[0] == 'G') :
    continue
  if (gcheck(primerr)) :
    continue
  primerftemp = int(x[a * 6 + 3])
  probetemp = int(x[a * 6 + 4])
  primerrtemp = int(x[a * 6 + 5])
  #temperature of probe must be 6-8 C higher than primer
  if (probetemp - primerftemp < 6 or probetemp - primerftemp > 8 or probetemp - primerrtemp < 6 or probetemp - primerrtemp > 8 ):
    continue
  print(primerf + " " + probe + "  " + primerr)


In [11]:
#Input: The sequence should be put in a file called sequence.txt.
#Output: The changed sequence will be put in a file called newsequence.txt.
#Find the data folder from the folder tab on the left tool bar.
#Both files will have to be created outside of the sample data folder.
#input is a sequence that contains cpg islands for a biomarker
#output is a list of amplicons from the original sequence that could contain forward and reverse primers
fin = open ('sequence.txt', 'r')
fout = open ('newsequence.txt', 'w')
x = fin.readlines()
seq = ""
for i in range (len(x)):
  seq += x[i]
newseq = ""
used = False
forward = -1
seq2 = seq
#seq = complementary(seq)
#convert c to t in an unmethylated strand
for i in range (len(seq)):
  if (seq[i] == 'C' or seq[i] == 'c') :
    if (i + 1 >= len(seq)):
      if (seq[i] == 'C'):
         newseq += 'T'
      else:
        newseq += 't'
    elif (seq[i + 1] == 'G' or seq[i + 1] == 'g'):
      newseq += seq[i]
    else:
      if (seq[i] == 'C'):
        newseq += 'T'
      else:
        newseq += 't'
  else:
    if (seq[i] == 'T' or seq[i] == 't' or seq[i] == 'G' or seq[i] == 'g' or seq[i] == 'A' or seq[i] == 'a'):
      newseq += seq[i]
print(newseq)
point = 0
start = 0
numsarr = []
sseqarr = []
first = True
while True:
  start = dense(newseq,start,first)
  first = False
  #filter out strands that dont have enough cpg pairs
  if (start == -1):
    if (len(numsarr) > 0):
      print()
      break
    print("INVALID")
    break
  sseq = newseq[start:start + 150]
  #print(start)
  #print(sseq)



  #print("W",sseq)
  places = cgcgpairs(sseq)
  #print(sseq)
  large = False
  for i in range (len(places)):
    if (places[i] == '-'):
      large = True
      break
  if (large):
    start += 1
    continue
  #print(sseq)
  #print("temp",temp(sseq[splace:eplace + 2]))
  #print(score)
  #print(gcdensity(sseq[splace:eplace + 2]))
  numsarr.append([score,start,splace,eplace,sseq,sseq[splace:eplace + 2],splace2,eplace2])
  #numsarr.append([score,splace,eplace])
  #print(numsarr[len(numsarr) - 1][0])
  #sseqarr.append([sseq,sseq[splace:eplace + 2]])
  #print(splace,eplace + 2)
  #print(start)
  start += 1
  #break

#do everything for the reverse strand
print(newseq)
point = 0
start = 0
sseq2arr = []
first = True
while True:
  start = dense(newseq,start,first)
  first = False
  if (start == -1):
    if (len(numsarr) > 0):
      print()
      break
    print("INVALID")
    break
  sseq2 = newseq[start:start + 150]
  #print(start)
  #print(sseq2)
  g3 = []
  g4 = False
  #print("W",sseq2,start)
  for i in range (20,130):
    if (sseq2[i:i + 4] == 'gggg' or sseq2[i:i + 4] == "GGGG"):
      start += i + 1;
      g4 = True
      break
    if (sseq2[i:i + 3] == "ggg" or sseq2[i:i + 3] == "GGG"):
      g3.append(i)
  for i in range (len(g3) - 1):
    if (g3[i + 1] - g3[i] <= 10):
      start += g3[i] + 1
      g4 = True
      break;
  if(g4):
    continue
  #print("W",sseq2)
  places = cgcgpairs(sseq2)
  #print(sseq2)
  large = False
  for i in range (len(places)):
    if (places[i] == '-'):
      large = True
      break
  if (large):
    start += 1
    continue
  #print(sseq2)
  #print("temp",temp(sseq2[splace:eplace + 2]))
  #print(score)
  #print(gcdensity(sseq2[splace:eplace + 2]))
  numsarr.append([score,start,splace,eplace,sseq2,sseq2[splace:eplace + 2],splace2,eplace2])
  #numsarr.append([score,splace,eplace])
  #print(numsarr[len(numsarr) - 1][0])
  #sseq2arr.append([sseq2,sseq2[splace:eplace + 2]])
  #print(splace,eplace + 2)
  #print(start)
  start += 1
  #break

#sort all primer candidates by score and output
numsarr.sort()
lastp = -2
lastsub = "-1"
for i in range(len(numsarr) - 1,-1,-1):
  if (numsarr[i][1] == lastp - 1 and numsarr[i][5] == lastsub):
      lastp -= 1
      continue
  elif (numsarr[i][1] == lastp and numsarr[i][5] == lastsub):
      continue
  else:
    lastp = numsarr[i][1]
    lastsub = numsarr[i][5]
  fout.write("Place in original sequence " + str(numsarr[i][1] + 1) + " to " + str(numsarr[i][1] + 150) + "\n")
  fout.write("Full sequence: " + str(complementary(numsarr[i][4])) + "\n")
  #fout.write("Start and End points: " + str(numsarr[i][6] + 1) + " " + str(numsarr[i][7] + 1) + "\n")
  #fout.write("Subsequence: " + complementary(numsarr[i][5]) + "\n")
  #print(numsarr[i][6])
  #print("Forward primer:",numsarr[i][4][numsarr[i][6] - 19:numsarr[i][6] + 1])
  #print("Reverse primer thing:",complementary(numsarr[i][4][numsarr[i][7]:numsarr[i][7] + 20]))
  print()
#print(newseq[])
fout.close()

CGCGCGCGGAGTTCGTTGAGATTTGAATTAATTTGGTTTAACGGTTTTTTTTAAATCGTTAGGAGTTTTTAATCGGCGGGATAGTAGGGCGCGGTGAGTTATCGTCGGTGATTAAGCGATTTTATTTTTTTTTTTCGGGTTTTTTTTTGTTATCGTCGTTTCGTAATTTTCGTCGTTCGAAGTTGGATTGAGTTCGTTAGGTTTTTCGATAGAATTTTTTTTTTTTTTAATATTTTTTCGTTAAGGTAAGTCGATGGATAGAGGCGCGGGTCGGAGTAGTTTTTTTTTTTAAGCGGGCGGCGCGCGAGGTTGCGGCGAGGTTTGAGTTTTGCGTTTTTGCGTTGTGCGCGTTTTTATTTCGCGTTTTAATTTTAGGCGTTTTTTGTTTTTTTTTTCGCGATTTTAGATTTGAGGGATTTTTTATTTTTTTTTTTTTTCGTTTTTTTGTTTGCGGGTTTTTTCGTTTGATCGTAGTTTCGAGATCGTCGCGTATTTTTTTTTACGTTTTTTTGGCGTGGTGTTATCGGATTTTTTTGGTTTAGTTTTAGGTGGATTTTTTTTTTATCGCGCGATTTCGTTTTTTTTAGTATTTTAGGGTGAGTTTAGTTTAGATTATTATTCGGAAAGTTTTTAAAAGTTTTAGTTTAGCGTTGAAGTAACGGGATTATGTTTAGTTTTAGGTTTCGGAGTAGGAAGGTTTGAGGGCGTTTTTATTTTATTCGTTTATTTTTTTCGTTTTTCGTTAGGTTTTTATTGGTTGGCGCGTTTCGCGGTTGGGATGGTAGTGGGAGGGGATTTTTTTTTTTAACGGGGTTATAAAAATAGCGTTTTCGGCGGGGTTTAGTTTTTTGTTATTTTCGTTTCGAGGTTTTCGCGTTAGAGACGTAGTCGCGTTTTTATTATTTATATTTATCGCGTTTTCGTTCGTTTTTTTTTCGGGAGTTAGTTCGCGTTATCGTCGTCGTTTAGGTTATCGTTATTTTTCGTAGTTATGTTTATT

next steps:

1.   download ucsc database of gene and parsing
2.   score system remodeling
3.   summarize rules
4.   allow code to check top and bottom strand
5.   make it into a usable multiplex assay

